In [ ]:
import pandas as pd
import datetime as dt
import holidays
import re
import json

# Define sets of people
set1 = ["jesca", "priscilla"]
set2 = ["Joseph", "sam", "shelton"]
set3 = ["Norbat", "Geoffrey", "antony", "James G", "james M", "Edwin", "Alphonse"]

# Function to get suitable people for a given shift
def suitable_people(day_type, shift):
    """
    day_type: "weekday", "weekend", or "holiday"
    shift: "A", "B", or "C"
    returns: list of possible people for that shift
    """

    if day_type == "weekday":
        if shift == "A":
            return set1 + set3   # weekday A → set1 + set3
        elif shift in ["B", "C"]:
            return set2 + set3   # weekday B/C → set2 + set3
        else:
            return []

    elif day_type in ["weekend", "holiday"]:
        if shift == "A":
            return set2 + set3   # weekend/holiday A → set2 + set3
        elif shift in ["B", "C"]:
            return set2 + set3   # weekend/holiday B/C → set2 + set3
        else:
            return []

    return []


# Function to categorize today
def get_today_category(today_date):
    kenya_holidays = holidays.Kenya()
    day_name = today_date.strftime("%A")

    if today_date in kenya_holidays:
        holiday_name = kenya_holidays.get(today_date)
        return "holiday"
    elif day_name in ["Saturday", "Sunday"]:
        return "weekend"
    else:
        return "weekday"


# Function to get suggested people for a shift
def assign_shift(shift, today_date):
    day_type = get_today_category(today_date)
    possible_people = suitable_people(day_type, shift)
    return possible_people


# ---------------- Example usage ----------------
# input_date should be a string in "YYYY-MM-DD" format
input_date = "2025-08-0"# <-- change date here if needed
today = dt.datetime.strptime(input_date, "%Y-%m-%d").replace(hour=16, minute=0)

# formatted string
today_str = today.strftime("%A, %d-%m-%Y")


# print(json.dumps({"today": today_str}, indent=4))

for s in ["A", "B", "C"]:
 
#  print(f"Suggested for shift {s}:", assign_shift(s, today))
# group1 = suggested for shift A on weekdays
# group2 = suggested for shift B/C on weekdays
# group3 = suggested for A weekends shift on any days                   




 learningData = pd.read_csv("C:/Users/samy/Downloads/shift_schedule_august_2025.csv" )


def last_shift_info(learningData, input_date, set2, set3):
    end_date = dt.datetime.strptime(input_date, "%d-%m-%Y").replace(hour=16, minute=0)
    def _normalize_name(name):
        return name.strip().lower().replace(".", "")
    
    set2_set3_lower = set(_normalize_name(n) for n in set2 + set3)
    last_shift = {name: None for name in set2_set3_lower}

    learningData_dates = pd.to_datetime(learningData['Date'], dayfirst=True, errors="coerce")

    for idx, row in learningData.iterrows():
        row_date = learningData_dates[idx]
        if pd.isna(row_date):
            continue

        for shift_col, shift_end_time in [
            ('Shift 1 (08:00–16:00)', dt.time(16, 0)),
            ('Shift 2 (16:00–00:00)', dt.time(0, 0)),
            ('Shift 3 (00:00–08:00)', dt.time(8, 0))
        ]:
            names_cell = row.get(shift_col)
            if pd.isna(names_cell):
                continue

            for name in str(names_cell).split(','):
                name_clean = _normalize_name(name)
                if name_clean in set2_set3_lower:
                    
                    shift_datetime = dt.datetime.combine(row_date.date(), shift_end_time)

                    if shift_col == 'Shift 2 (16:00–00:00)':
                   
                        shift_datetime = dt.datetime.combine(row_date.date(), dt.time(23, 59))
                    elif shift_col == 'Shift 3 (00:00–08:00)':
                        shift_datetime += dt.timedelta(days=1)

                    if shift_datetime <= end_date:
                        if last_shift[name_clean] is None or shift_datetime > last_shift[name_clean]:
                            last_shift[name_clean] = shift_datetime

    # Convert dict → DataFrame
    data = []
    for name, shift_time in last_shift.items():
        if shift_time:
            hours_since = (end_date - shift_time).total_seconds() / 3600
        else:
            hours_since = None
        data.append({
            "Name": name.capitalize(),
            "Last Shift End": shift_time,
            "Hours Since Last Shift": hours_since
        })

    result_df = pd.DataFrame(data).sort_values(by="Hours Since Last Shift", na_position="last")
    return result_df


def last_shift_infoA(learningData, today, set2, set3):
    end_date = (today - dt.timedelta(hours=8))

    def _normalize_name(name):
        return name.strip().lower().replace(".", "")
    
    set2_set3_lower = set(_normalize_name(n) for n in set2 + set3)
    all_shifts = {name: [] for name in set2_set3_lower}

    learningData_dates = pd.to_datetime(learningData['Date'], dayfirst=True, errors="coerce")

    for idx, row in learningData.iterrows():
        row_date = learningData_dates[idx]
        if pd.isna(row_date):
            continue

        for shift_col, shift_end_time in [
            ('Shift 1 (08:00–16:00)', dt.time(16, 0)),
            ('Shift 2 (16:00–00:00)', dt.time(23, 59)),  # ✅ always 23:59
            ('Shift 3 (00:00–08:00)', dt.time(8, 0))
        ]:
            names_cell = row.get(shift_col)
            if pd.isna(names_cell):
                continue

            for name in str(names_cell).split(','):
                name_clean = _normalize_name(name)
                if name_clean in set2_set3_lower:
                    shift_datetime = dt.datetime.combine(row_date.date(), shift_end_time)

                    if shift_col == 'Shift 3 (00:00–08:00)':
                        shift_datetime += dt.timedelta(days=1)

                    if shift_datetime <= end_date:
                        all_shifts[name_clean].append(shift_datetime)

    # ✅ Always pick the latest shift ≤ cutoff
    data = []
    for name, shifts in all_shifts.items():
        shifts_sorted = sorted(shifts)
        last_valid = shifts_sorted[-1] if shifts_sorted else None

        if last_valid:
            hours_since = (end_date - last_valid).total_seconds() / 3600
        else:
            hours_since = None

        data.append({
            "Name": name.capitalize(),
            "Last Shift End": last_valid,
            "Hours Since Last Shift": hours_since
        })

    return pd.DataFrame(data).sort_values(by="Hours Since Last Shift", na_position="last")
last_info_df = last_shift_infoA(learningData, today, set2, set3)



def predict_shiftA_weekday(learningData, today, last_shift_infoA):    
    day_type = get_today_category(today)
    if day_type != "weekday":
        return {
            "message": "Shift A weekday prediction is only for weekdays",
            "graded_set3": [],
            "set3_rest_info": {},
            "df_result": None
        }

    last_info_df = last_shift_infoA(learningData, today, set2, set3)

    # Convert to dict for lookup: name → hours since last shift
    last_shift_dict = (
        last_info_df
        .assign(Name=lambda df: df["Name"].str.strip().str.lower())
        .set_index("Name")["Hours Since Last Shift"]
        .to_dict()
    )

    rest_info = {}
    graded_people = set()

    for person in set3:
        key1 = person.lower()
        key2 = person.capitalize()
        hours_rest = last_shift_dict.get(key2) or last_shift_dict.get(key1)

        if hours_rest is None:
            grade = 100
        else:
            if hours_rest >= 32:
                grade = 100
            elif hours_rest >= 24:
                grade = 75
            elif hours_rest >= 16:
                grade = 50
            else:
                grade = 0

        rest_info[person] = {"hours_rest": hours_rest, "grade": grade}
        if grade > 0:
            graded_people.add(person)

    # ✅ Build DataFrame
    df_result = pd.DataFrame.from_dict(rest_info, orient="index").reset_index()
    df_result.rename(columns={"index": "Name"}, inplace=True)

    return {
        "message": "Success",
        "graded_set3": list(graded_people),
        "set3_rest_info": rest_info,
        "df_result": df_result
    }

def most_suitable_people(learningData, today, graded_people, rest_info, set3):
    def _normalize_name(name):
        return name.strip().lower().replace(".", "")

    # Window: Monday 00:00 → Today 07:59
    start_of_week = today - dt.timedelta(days=today.weekday())
    start_of_week = start_of_week.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_lookback = today.replace(hour=7, minute=59, second=59, microsecond=0)

    # Normalize set3 for filtering
    set3_lower = {_normalize_name(p) for p in set3}

    # Track set3 members who worked Shift A
    worked_shiftA = set()

    learningData_dates = pd.to_datetime(learningData['Date'], dayfirst=True, errors="coerce")

    for idx, row in learningData.iterrows():
        row_date = learningData_dates[idx]
        if pd.isna(row_date):
            continue

        shift_end = dt.datetime.combine(row_date.date(), dt.time(16, 0))

        if not (start_of_week <= shift_end <= end_of_lookback):
            continue

        names_cell = row.get('Shift 1 (08:00–16:00)')
        if pd.isna(names_cell):
            continue

        for name in str(names_cell).split(','):
            name_clean = _normalize_name(name)
            if name_clean in set3_lower:   # ✅ only keep set3 members
                worked_shiftA.add(name_clean)

    # Suitable = graded set3 who are not in worked_shiftA
    most_suitable = [
        p for p in graded_people
        if _normalize_name(p) not in worked_shiftA and _normalize_name(p) in set3_lower
    ]

    return {
        "most_suitable": most_suitable,
        "worked_shiftA": list(worked_shiftA),
        "rest_info": {p: rest_info[p] for p in most_suitable}
    }


# 1. Run prediction
prediction = predict_shiftA_weekday(learningData, today, last_shift_infoA)

# 2. Extract graded people and rest info
graded_people = prediction["graded_set3"]
rest_info = prediction["set3_rest_info"]

# 3. Find most suitable
results = most_suitable_people(learningData, today, graded_people, rest_info,set3)


def predict_shiftB(learningData, today, last_shift_df):
    set2_set3_lower = set(n.lower() for n in set2 + set3)
    last_shift_information = last_shift_df.set_index("Name").to_dict(orient="index")

    most_suitable = set()
    rest_grades = {}
    rows = []  # to collect for dataframe

    for name, info in last_shift_information.items():
        person = name.lower()
        if person not in set2_set3_lower:
            continue  # skip people not in set2+set3

        hours_rest = info.get("Hours Since Last Shift", 0)

        # Grading logic
        if hours_rest is None:
            grade = 100
        elif hours_rest >= 32:
            grade = 100
        elif hours_rest >= 24:
            grade = 75
        elif hours_rest >= 16:
            grade = 50
        else:
            grade = 0

        # Eligibility
        eligible = (hours_rest is None) or (hours_rest >= 16)
        if eligible:
            most_suitable.add(name)

        # Store results
        rest_grades[name] = {
            "hours_rest": hours_rest,
            "grade": grade,
            "eligible": eligible
        }

        # Collect row for dataframe
        rows.append({
            "Name": name,
            "Last Shift End": info.get("Last Shift End"),
            "Hours Rested": hours_rest,
            "Grade %": grade,
            "Eligible": eligible
        })

    # Handle people in set2+set3 never seen in last_shift_info
    for person in set2_set3_lower:
        names_in_df = [n.lower() for n in last_shift_information.keys()]
        if person not in names_in_df:
            rest_grades[person] = {"hours_rest": None, "grade": 100, "eligible": True}
            most_suitable.add(person)
            rows.append({
                "Name": person,
                "Last Shift End": None,
                "Hours Rested": None,
                "Grade %": 100,
                "Eligible": True
            })
            print(f" - {person}: never worked → grade 100%, ELIGIBLE")

    # Convert results into DataFrame
    result_df = pd.DataFrame(rows).sort_values(by=["Eligible", "Grade %", "Hours Rested"], ascending=[False, False, False])

    return {
        "most_suitable ShiftB": list(most_suitable),
        "set2_set3_rest_grades ShiftB": rest_grades,
        "result_df": result_df
    }
results = predict_shiftB(
    learningData,
    today,
    last_shift_info(learningData, today.strftime("%d-%m-%Y"), set2, set3)
)



def predict_shiftC(learningData, input_date, set2, set3):
    # Parse cutoff datetime → 00:00 (midnight)
    end_date = (dt.datetime.strptime(input_date, "%d-%m-%Y") + dt.timedelta(days=1)).replace(hour=0, minute=0)


    # Normalize names
    def _normalize_name(name):
        return name.strip().lower().replace(".", "")
    
    set2_set3_lower = set(_normalize_name(n) for n in set2 + set3)
    last_shift = {name: None for name in set2_set3_lower}

    learningData_dates = pd.to_datetime(learningData['Date'], dayfirst=True, errors="coerce")

    for idx, row in learningData.iterrows():
        row_date = learningData_dates[idx]
        if pd.isna(row_date):
            continue

        for shift_col, shift_end_time in [
            ('Shift 1 (08:00–16:00)', dt.time(16, 0)),
            ('Shift 2 (16:00–00:00)', dt.time(0, 0)),
            ('Shift 3 (00:00–08:00)', dt.time(8, 0))
        ]:
            names_cell = row.get(shift_col)
            if pd.isna(names_cell):
                continue

            for name in str(names_cell).split(','):
                name_clean = _normalize_name(name)
                if name_clean in set2_set3_lower:
                    
                    # Default shift datetime
                    shift_datetime = dt.datetime.combine(row_date.date(), shift_end_time)

                    # ✅ Special handling
                    if shift_col == 'Shift 2 (16:00–00:00)':
                        shift_datetime = dt.datetime.combine(row_date.date(), dt.time(23, 59))
                    elif shift_col == 'Shift 3 (00:00–08:00)':
                        shift_datetime += dt.timedelta(days=1)

                    # ✅ Only include shifts that ended before/equal to 00:00 cutoff
                    if shift_datetime <= end_date:
                        if last_shift[name_clean] is None or shift_datetime > last_shift[name_clean]:
                            last_shift[name_clean] = shift_datetime

    # Convert dict → DataFrame
    data = []
    for name, shift_time in last_shift.items():
        if shift_time:
            hours_since = (end_date - shift_time).total_seconds() / 3600
        else:
            hours_since = None
        data.append({
            "Name": name.capitalize(),
            "Last Shift End": shift_time,
            "Hours Since Last ShiftC": hours_since
        })

    result_df = pd.DataFrame(data).sort_values(by="Hours Since Last ShiftC", na_position="last")
    return result_df   
shiftC_df = predict_shiftC(
    learningData, 
    today.strftime("%d-%m-%Y"), 
    set2, 
    set3
)



def predict_shiftA_weekend_holiday(learningData, today):    
    day_type = get_today_category(today)
    if day_type not in ["weekend", "holiday"]:
        return "Shift A weekend/holiday prediction is only for weekends and holidays"

    # Lookback window
    start_of_lookback = today - dt.timedelta(days=8)
    end_of_lookback = today - dt.timedelta(days=1)

    # Parse dates in dataset
    learningData_dates = pd.to_datetime(learningData['Date'], dayfirst=True, errors="coerce")

    # Filter last 14 days
    lookback_data = learningData[(learningData_dates >= start_of_lookback) &
                                 (learningData_dates <= end_of_lookback)]

    # Collect people who already worked Shift A (weekends/holidays)
    set2_set3_lower = set(n.lower() for n in set2 + set3)
    worked_shiftA_weekend_holiday = set()
    last_shift_time = {}

    for idx, row in lookback_data.iterrows():
        row_date = pd.to_datetime(str(row['Date']).strip(), dayfirst=True, errors='coerce')
        if pd.isna(row_date):
            continue
        row_day_name = row_date.strftime("%A")
        is_holiday = row_date in holidays.Kenya()
        if row_day_name in ["Saturday", "Sunday"] or is_holiday:
            names_cell = row.get('Shift 1 (08:00–16:00)')
            if pd.isna(names_cell):
                continue
            for name in str(names_cell).split(','):
                name_clean = name.strip().lower()
                if name_clean in set2_set3_lower:
                    worked_shiftA_weekend_holiday.add(name_clean)
                    # Track the most recent time they worked
                    shift_datetime = dt.datetime.combine(row_date.date(), dt.time(16, 0))  # Shift A ends 16:00
                    if name_clean not in last_shift_time or shift_datetime > last_shift_time[name_clean]:
                        last_shift_time[name_clean] = shift_datetime
                        # print("lastshift infor c: ", name_clean, shift_datetime)

    # Eligible people (ignoring history)
    suitable_for_shiftA_wh = set(n.lower() for n in suitable_people(day_type, "A"))

    # Exclude those who worked Shift A recently
    available_people = suitable_for_shiftA_wh - worked_shiftA_weekend_holiday

    # print("Worked last two weekends", worked_shiftA_weekend_holiday)



    last_shift_dict = last_info_df.set_index(last_info_df["Name"].str.lower())["Last Shift End"].to_dict()
    # print("last info: ", last_shift_dict)

    most_suitable = set()
    rest_info = {}
  

    for person in available_people:
        name_clean = person.lower()
        last_time = last_shift_dict.get(name_clean)

        if last_time is None:
        # Never worked → full grade
            hours_rest = None
            grade = 100
            most_suitable.add(person)
        else:
        # Compute hours since last shift (reference 08:00 today)
           reference_time = dt.datetime.combine(today.date(), dt.time(8, 0))
           hours_rest = (reference_time - last_time).total_seconds() / 3600

        # Assign grade
           if hours_rest >= 32:
                  grade = 100
           elif hours_rest >= 24:
               grade = 75
           elif hours_rest >= 16:
               grade = 50
           else:
               grade = 0

           if grade > 0:
               most_suitable.add(person)

    # Store info for printing or DataFrame
        rest_info[person] = {"hours_rest": hours_rest, "grade": grade}
    result_most_suitable = {
    person: rest_info[person] for person in most_suitable
        }
    resulting_available_people= {
    person: rest_info[person] for person in available_people
        }
        

# Convert to DataFrame if needed
    df_rest = pd.DataFrame.from_dict(rest_info, orient="index").reset_index()
    df_rest.rename(columns={"index": "Name"}, inplace=True)

    # print("Lookback range:", start_of_lookback.date(), "to", end_of_lookback.date())
    # print("Worked Shift A on weekends/holidays in last 14 days:", worked_shiftA_weekend_holiday)
    # print("All suitable for Shift A on weekends/holidays:", suitable_for_shiftA_wh)
    # print("Available (excluded recent workers):", resulting_available_people)
    # print("Most suitable (rested ≥24h):", result_most_suitable)

    return {
        "most_suitable": (result_most_suitable),
        "fallback_suitable": (resulting_available_people)
    }

prediction = predict_shiftA_weekend_holiday(learningData, today)



import json
import datetime as dt
from decimal import Decimal

# optional imports (safe if absent)
try:
    import pandas as pd
except Exception:
    pd = None

try:
    import numpy as np
except Exception:
    np = None


def dict_to_array(rest_info_dict):
    """Convert {name: {...}} -> [ {name: name, ...}, ... ]"""
    return [
        {"name": name, **(info if isinstance(info, dict) else {"value": info})}
        for name, info in rest_info_dict.items()
    ]


def safe_to_json(obj):
    """
    Convert common pandas/numpy/datetime objects into JSON-serializable Python types.
    Works recursively for lists/dicts/tuples/sets.
    """
    # pandas DataFrame -> list of records
    if pd is not None and isinstance(obj, pd.DataFrame):
        records = obj.to_dict(orient="records")
        return [safe_to_json(r) for r in records]

    # pandas Series -> dict
    if pd is not None and isinstance(obj, pd.Series):
        return {k: safe_to_json(v) for k, v in obj.to_dict().items()}

    # pandas / numpy "missing" values -> None
    try:
        if pd is not None and pd.isna(obj):
            return None
    except Exception:
        pass

    # pandas Timestamp
    if pd is not None and isinstance(obj, pd.Timestamp):
        return obj.isoformat()

    # native datetimes / dates
    if isinstance(obj, dt.datetime) or isinstance(obj, dt.date):
        return obj.isoformat()

    # numpy scalar -> python scalar
    if np is not None and isinstance(obj, np.generic):
        return obj.item()

    # Decimal -> float
    if isinstance(obj, Decimal):
        return float(obj)

    # dict -> recursively convert
    if isinstance(obj, dict):
        return {k: safe_to_json(v) for k, v in obj.items()}

    # list/tuple/set -> list
    if isinstance(obj, (list, tuple, set)):
        return [safe_to_json(v) for v in obj]

    # primitives (int/float/str/bool/None) are OK
    if isinstance(obj, (str, int, float, bool)) or obj is None:
        return obj

    # fallback
    return str(obj)


def build_final_output(learningData, today, last_shift_infoA, set2, set3):
    # ensure today is datetime
    if isinstance(today, str):
        try:
            today = dt.datetime.strptime(today, "%Y-%m-%d").replace(hour=16, minute=0)
        except Exception:
            today = dt.datetime.fromisoformat(today)

    today_str = today.strftime("%d-%m-%Y")

    # --- Shift A (weekday) ---
    predictionA = predict_shiftA_weekday(learningData, today, last_shift_infoA)
    if isinstance(predictionA, dict):
        graded_people = predictionA.get("graded_set3", [])
        rest_info = predictionA.get("set3_rest_info", {})
        resultsA = most_suitable_people(learningData, today, graded_people, rest_info, set3)
        rest_info_array = dict_to_array(resultsA.get("rest_info", {}))
    else:
        rest_info_array = []

    # --- Shift B ---
    resultsB = predict_shiftB(
        learningData,
        today,
        last_shift_info(learningData, today.strftime("%d-%m-%Y"), set2, set3)
    )
    shiftB_data = resultsB.get("result_df", [])

    # --- Shift C ---
    resultsC = predict_shiftC(learningData, today.strftime("%d-%m-%Y"), set2, set3)
    resultsC_array = dict_to_array(resultsC) if isinstance(resultsC, dict) else resultsC

    # --- Shift A weekend/holiday ---
    resultsD = predict_shiftA_weekend_holiday(learningData, today)
    most_suitable, fallback = [], []
    if isinstance(resultsD, dict):
        most_suitable = dict_to_array(resultsD.get("most_suitable", {}))
        fallback = dict_to_array(resultsD.get("fallback", {}))
    elif isinstance(resultsD, list):
        most_suitable = resultsD

    # ✅ Final output: independent items
    final_output = {
    "ShiftA": safe_to_json(rest_info_array),
    "ShiftB": safe_to_json(shiftB_data),
    "ShiftC": safe_to_json(resultsC_array),
    "ShiftAW": safe_to_json(most_suitable),
    "ShiftAWF": safe_to_json(fallback),
}

    
    


    print("final json output:", json.dumps(final_output, indent=2))
    return [final_output]




output = build_final_output(
    learningData=learningData,                # your real DataFrame
    today=today,
    last_shift_infoA=last_shift_infoA,        # pass the function
    set2=set2,                                # your real set2 list
    set3=set3                                 # your real set3 list
)

print("\nReturned to n8n:", json.dumps(output, indent=2))


# n8n will automatically capture this return
# return build_final_output(learningData, today, last_shift_infoA, set2, set3)

SyntaxError: invalid syntax (4116740638.py, line 62)